## Ontario research project fund predictor 

The outline of this notebool includes:
* 1. import the data
* 2. split the data into train and test set
* 3. make tf-idf for training dataset and use it for test set 
* 4. build a logistic regression on training tfidf data 
* 5. evaluate the model

First, lets download all the required packages. 


In [1]:
import os
import sys
import pandas as pd
import numpy as np
import nltk

from pyprojroot import here
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import classification_report
from joblib import dump,load

# 1. Import the data

I imported the data which was cleaned in data_cleaning.ipynb.

In [2]:
# Read in the data in the previous step
df = pd.read_pickle('..\data\processed\data_clean.pkl')
len(df)

3059

In [3]:
df.project_title[0]

'density matrix renormalization group simulations of systems in condensed matter physics and quantum chemistry'

# 2. Split the data to train-test set

Here the data was splited to train and test set. 

In [4]:
y6=df['two_labela_ontario_commitment']
x=df['project_title']

In [5]:
def split_data(x,y):
    x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)
    return (x_train,x_test,y_train,y_test)

In [6]:
x_train,x_test,y_train6,y_test6 =split_data(x,y6)

In [7]:
len(x_train),len(x_test)

(2294, 765)

In [8]:
type(x_train)

pandas.core.series.Series

# 3.1. Make Tf-idf for training

I used the TfidfVectorizer to convert the words to matrix of TF_IDF features.

In [9]:
#nltk.download('stopwords')

In [10]:
tfidfvectorizer = TfidfVectorizer(analyzer='word', lowercase=True, max_df=0.9,min_df=2,ngram_range=(1,1),stop_words='english')

In [11]:
tfidfvectorizer.fit(x_train)
tfidf_train = tfidfvectorizer.transform(x_train)

In [12]:
tfidf_test  = tfidfvectorizer.transform(x_test)

In [13]:
#print(tfidf_train)

In [14]:
test=["Laboratory for Forest Genomics"]
test1=tfidfvectorizer.transform(test)
print(test1)

  (0, 804)	0.3029169163878714
  (0, 624)	0.5898837280626423
  (0, 592)	0.7485175543252023


In [15]:
test1

<1x1644 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [16]:
tfidf_train.shape,tfidf_test.shape

((2294, 1644), (765, 1644))

In [17]:
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
count_wm=countvectorizer.fit_transform(x_train)

In [18]:
count_tokens = countvectorizer.get_feature_names()
tfidf_tokens = tfidfvectorizer.get_feature_names()

C:\Users\Atieh\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [19]:
tfidf_train.shape, tfidf_test.shape

((2294, 1644), (765, 1644))

In [20]:
len(tfidf_tokens)

1644

In [21]:
len(set(tfidf_tokens))

1644

In [22]:
tfidf_tokens[100:]

['assessing',
 'assessment',
 'assistance',
 'assistive',
 'associated',
 'assurance',
 'asthma',
 'astronomical',
 'astronomy',
 'astrophysics',
 'asymmetric',
 'atherothrombotic',
 'atmosphere',
 'atmospheric',
 'atomic',
 'atoms',
 'attention',
 'attosecond',
 'audiology',
 'audiovisual',
 'auditory',
 'augmented',
 'autism',
 'autoimmunity',
 'automated',
 'automation',
 'automotive',
 'autonomic',
 'autonomous',
 'avian',
 'aviation',
 'bacteria',
 'bacterial',
 'bacteriophages',
 'balance',
 'barriers',
 'based',
 'basic',
 'basis',
 'beam',
 'bed',
 'behavior',
 'behavioral',
 'behaviour',
 'behavioural',
 'benefit',
 'beta',
 'better',
 'beverage',
 'big',
 'bilingual',
 'bioactive',
 'bioarchaeology',
 'biochemical',
 'biochemistry',
 'biodegradable',
 'biodiversity',
 'bioelectrochemical',
 'bioengineering',
 'biofilm',
 'biofuels',
 'biogeochemistry',
 'bioimaging',
 'bioinformatic',
 'bioinformatics',
 'biological',
 'biology',
 'biomanufacturing',
 'biomarker',
 'biomarker

In [23]:
#print(tfidf_train) 

In [24]:
len(list(tfidfvectorizer.vocabulary_.keys())),len(set(list(tfidfvectorizer.vocabulary_.keys())))

(1644, 1644)

# 4. Model

# 4.1 Logistic regression on two categories prediction and tf-idf 

In [25]:
y_train6.shape, tfidf_train.shape

((2294,), (2294, 1644))

In [26]:
type(tfidf_train)

scipy.sparse.csr.csr_matrix

In [27]:
# logistic regression
logistic_model = LogisticRegression()
logistic_model.fit(tfidf_train, y_train6)

LogisticRegression()

In [28]:
logistic_model.score(tfidf_train,y_train6)

0.8112467306015693

In [29]:
logistic_model.score(tfidf_test,y_test6)

0.5973856209150327

In [30]:
y_pred = logistic_model.predict(tfidf_test)

In [31]:
accuracy = accuracy_score(y_test6,y_pred)*100
accuracy

59.73856209150327

In [32]:
confusion_mat = confusion_matrix(y_test6,y_pred)
confusion_mat

array([[274, 134],
       [174, 183]], dtype=int64)

In [33]:
# Create a hyperparameter grid for LogisticRegression
log_reg_grid = {"C": np.logspace(-4, 4, 20)}
rs_log_reg = RandomizedSearchCV(LogisticRegression(),
                                param_distributions=log_reg_grid)

In [34]:
rs_log_reg.fit(tfidf_train, y_train6)

C:\Users\Atieh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Atieh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

RandomizedSearchCV(estimator=LogisticRegression(),
                   param_distributions={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04])})

In [35]:
rs_log_reg.best_params_

{'C': 0.08858667904100823}

# 5. Evaluate the model

In [36]:
rs_log_reg.score(tfidf_train, y_train6)

0.7057541412380122

In [37]:
y_preds6=rs_log_reg.predict(tfidf_test)
print(classification_report(y_test6, y_preds6))

              precision    recall  f1-score   support

           0       0.60      0.80      0.69       408
           1       0.63      0.38      0.48       357

    accuracy                           0.61       765
   macro avg       0.61      0.59      0.58       765
weighted avg       0.61      0.61      0.59       765



# Save the model

In [38]:
test=['Establishment of the Intelligent Visualization Laboratory']

In [39]:
with open('../models/model_tfidf.joblib', 'wb') as f:
    dump(tfidfvectorizer, f)

In [40]:
#with open('../models/model_tfidf2.joblib', 'rb') as f:
model_input=load(open('../models/model_tfidf.joblib', 'rb'))
#loaded_model =load(open(model_input, 'rb'))

In [41]:
test1=model_input.transform(test)
print(test1)

  (0, 1604)	0.6133324136785168
  (0, 804)	0.23902562855823747
  (0, 760)	0.5574365795286463
  (0, 523)	0.5059195183297708


In [42]:
with open('../models/model.joblib', 'wb') as f:
    dump(logistic_model, f)

In [43]:
model_logistic=load(open('../models/model.joblib', 'rb'))

In [44]:
pred=model_logistic.predict(test1)
pred

array([0])

In [45]:
if pred[0] == 0:
    print ('The fund you will expected is in range of $ 5000 to $ 150000')
else:
    print ('The fund you will expected is in range of $ 150000 to $ 14000000')

The fund you will expected is in range of $ 5000 to $ 150000


In [48]:
import sklearn
print(sklearn.__version__)

1.0.2


In [50]:
python_version()

NameError: name 'python_version' is not defined